The AIm of this analysis is to develop a strong image classifer using less training data.

In [116]:
import numpy as np
import scipy
import os
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,regularizers
from keras.preprocessing .image import ImageDataGenerator
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD
from matplotlib import pyplot as plt   
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import load_model
from keras import regularizers
import cv2
#FinalModel.add(modelNew)
from keras import regularizers
from keras import layers
from keras.callbacks import *


Our training and validation directories will have following structure. Inspiration for this notebook comes from this [Keras blog post](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) and the VGG ConvNet paper.
There are training and test directories and have their corresponding directories as per image class categories. we will use two sets of pictures, which we got from [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data): 1000 cats and 1000 dogs (although the original dataset had 12,500 cats and 12,500 dogs, we just took the first 1000 images for each class). We will also use 400 additional samples from each class as validation data, to evaluate our model performance.
This is quite common in real-time world where we don't get too many data and hence we need to ensure our model is well trained with such minimal data.

We will first define the training and testing directory path and store all the images names of categories into their resoective arrasy.

In [149]:
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
TRAIN_DIR = '../input/train/'
TRAIN_DIR_DOG = '../input/train/dogs'
TRAIN_DIR_CATS = '../input/train/cats'
VALIDATION_DIR = '../input/validation/'

TEST_DIR = '../input/test/'

print("Path Is:",TRAIN_DIR)

sys.path.append(os.path.join(ROOT_DIR, EXTENSION_PATH))  # To find local version

train_dogs =   [TRAIN_DIR_DOG+inputD for inputD in os.listdir(TRAIN_DIR_DOG) if 'dog' in inputD]
train_cats =   [TRAIN_DIR_CATS+inputC for inputC in os.listdir(TRAIN_DIR_CATS) if 'cat' in inputC]
test_images =  [TEST_DIR+inputT for inputT in os.listdir(TEST_DIR) ]


print("No of Training Images of Dog:",len(train_dogs))
print("No of Training Images of Cats:",len(train_cats))
print("No of Test Images:",len(test_images))

C:\Users\akayal\Devapps
Path Is: ../input/train/
No of Training Images of Dog: 11252
No of Training Images of Cats: 11733
No of Test Images: 12500


We will now do basic preprocessing now as part of data loading. Data will be first resized and then will be normalised.

Keras provides the ImageDataGenerator class that defines the configuration for image data preparation and augmentation. Some of these capabailities are:

- Sample-wise standardization.
- Feature-wise standardization.
- ZCA whitening.
- Random rotation, shifts, shear and flips.
- Dimension reordering.
- Save augmented images to disk.

In [150]:
random.shuffle(train_dogs)
random.shuffle(train_cats)
random.shuffle(test_images)

Our next step will be to read the test and training images. Remember that training data are labelled ones and test data as usual does not have labelled ones.

In [151]:
image_height = 64
image_width=64

In [165]:
# we create two instances with the same arguments
data_gen_args = dict(
        #rotation_range=10,
       # width_shift_range=0.1,
       # height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


image_datagen_prep = ImageDataGenerator(**data_gen_args)

print("Path Is:",TRAIN_DIR)
train_generator_data = image_datagen_prep.flow_from_directory(directory = TRAIN_DIR,
                                                             target_size=(image_width,image_height),
                                                             classes=['dogs','cats'],
                                                             class_mode='binary',
                                                             batch_size=16,
                                                             color_mode='rgb')

validation_generator_data = image_datagen_prep.flow_from_directory(directory = VALIDATION_DIR,
                                                             target_size=(image_width,image_height),
                                                             classes=['dogs','cats'],
                                                             class_mode='binary',
                                                             batch_size=16,
                                                             color_mode='rgb')
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
print(train_generator_data)
   

Path Is: ../input/train/
Found 22985 images belonging to 2 classes.
Found 2015 images belonging to 2 classes.


# Model Development
We will now start model development

In [180]:
model_name = "dogsvscats_6_conv_layers"



CNNModel = Sequential()

#convolution 1st layer
droprate=0.5
droprateRev=0.7
#convolution 1st layer
CNNModel.add(Conv2D(32,(3,3), activation='relu', strides=(1, 1), padding='same', input_shape=(64,64,3)))
#model.add(Conv2D(32,(3,1), activation='relu', strides=(1, 1), padding='valid',input_shape=input_24)#(input_24)
CNNModel.add(BatchNormalization())
#CNNModel.add(MaxPooling2D(pool_size=(4, 4)))
#CNNModel.add(Dropout(droprate))#3

#convolution 2nd layer
CNNModel.add(Conv2D(64,(3,3), padding='valid',activation='relu'))
CNNModel.add(BatchNormalization())
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))
#CNNModel.add(Dropout(droprate))#3

#convolution 3rd layer
CNNModel.add(Conv2D(128,(3,3), padding='valid',activation='relu'))
CNNModel.add(BatchNormalization())
CNNModel.add(MaxPooling2D())
CNNModel.add(Dropout(droprateRev))#3

#convolution 3rd layer
CNNModel.add(Conv2D(256,(3,3), padding='valid',activation='relu'))
CNNModel.add(BatchNormalization())
CNNModel.add(MaxPooling2D())
CNNModel.add(Dropout(droprateRev))#3


#convolution 3rd layer
CNNModel.add(Conv2D(64,(1,1), activation='relu'))
CNNModel.add(BatchNormalization())
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))
CNNModel.add(Dropout(droprateRev))#3

#Fully connected final layer
CNNModel.add(Flatten())
   
CNNModel.add(Dense(100, activation='relu'))
CNNModel.add(Dropout(droprateRev))

CNNModel.add(Dense(50, activation='relu',use_bias=False, kernel_initializer='glorot_uniform'))
CNNModel.add(Dropout(droprateRev))#3


CNNModel.add(Dense(1, activation='sigmoid'))


In [182]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.002,
                              patience=2, min_lr=0.001)

In [183]:
CNNModel.compile(loss='binary_crossentropy',
              optimizer= 'RMSprop',
              metrics=['accuracy'])
CNNModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_116 (Conv2D)          (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_115 (Bat (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 62, 62, 64)        18496     
_________________________________________________________________
batch_normalization_116 (Bat (None, 62, 62, 64)        256       
_________________________________________________________________
max_pooling2d_90 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 29, 29, 128)       73856     
_________________________________________________________________
batch_normalization_117 (Bat (None, 29, 29, 128)       512       
__________

In [ ]:
history = CNNModel.fit_generator(generator=train_generator_data, 
                        steps_per_epoch=150,
                        epochs=15,
                        verbose=1,
                        validation_steps=30,  
                        validation_data=validation_generator_data,
                        callbacks=[reduce_lr])


Epoch 1/15
150/150 [==============================] - 398s 3s/step - loss: 2.1284 - acc: 0.4892 - val_loss: 0.9439 - val_acc: 0.4000
Epoch 2/15
150/150 [==============================] - 374s 2s/step - loss: 1.0744 - acc: 0.5171 - val_loss: 0.8378 - val_acc: 0.3937
Epoch 3/15
150/150 [==============================] - 381s 3s/step - loss: 0.8608 - acc: 0.5225 - val_loss: 0.9096 - val_acc: 0.3875
Epoch 4/15
131/150 [=========================>....] - ETA: 47s - loss: 0.7795 - acc: 0.5119

In [160]:
model = tflearn.DNN(convnet, tensorboard_dir="log")
if(os.path.exists("{}.meta".format(model_name))):
    model.load(model_name)
print("MODEL-LOADED")


NameError: name 'tflearn' is not defined

In [ ]:
#For doing predictions this worked for me:

from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

test_model = load_model('my_model_name.h5')
img = load_img('image_to_predict.jpg',False,target_size=(img_width,img_height))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = test_model.predict_classes(x)
prob = test_model.predict_proba(x)
print(preds, probs)

Using the bottleneck features of a pre-trained network. A more refined approach would be to leverage a network pre-trained on a large dataset. We will use the VGG16 architecture, pre-trained on the ImageNet dataset.